# Vibrational Spectra


## Wiedeholung: Harmonischer Oszillator

Ein quantenmechanisches Teilchen in einem parabolischen Potential ist ein harmonischer Oszillator:

$$
E_n = \hbar \omega \left( n + \frac{1}{2} \right) ~~~ \omega = \frac{\hbar k^2}{m} \\
\Psi_n(x') = \underbrace{\sqrt{ \frac{1}{2^n n! \sqrt{\pi} \alpha }}}_{\text{Normierungskoeffizient}~C_n} 
\cdot \underbrace{H_n(x')}_{\text{Hermit-Polynom}} 
\cdot \underbrace{e^-\frac{x'^2}{2}}_{\text{Gauss-Kurve}} ~~\text{mit}~~x'= x / \alpha ~~\text{mit } \alpha =  \left(\frac{\hbar^2}{mk}\right)^{\left(\frac{1}{4}\right)}
$$

Die sogenannten Hermitschen Polynome sind Lösungen spezieller Differentialgleichungen, die man nachschlagen kann. Die ersten drei Lösungen lauten:

$$
\begin{aligned}
E_0 = \frac{\hbar \omega}{2}  &\rightarrow \Psi_0(x') = \sqrt{ \frac{1}{\sqrt{\pi} }} e^-\frac{x'^2}{2} \\ 
E_1 = \frac{3 \hbar \omega}{2}  &\rightarrow \Psi_1(x') = \sqrt{ \frac{1}{2 \sqrt{\pi} }} 2x' e^-\frac{x'^2}{2} \\ 
E_2 = \frac{5 \hbar \omega}{2}  &\rightarrow \Psi_2(x') = \sqrt{ \frac{1}{8 \sqrt{\pi} }} \left( 4x'^2 -2 \right) e^-\frac{x'^2}{2} \\ 
\end{aligned}
$$

Der Übergang zwischen konsekutiven Energiezuständen ist spektroskopisch erlaubt, weil das Dipolmoment sich ändert. Das Integral $< \psi_{0} | \hat{\mu} | \psi_{1} >$ ist gerade.

Der Dipoloperator in einer Dimension in kartesischen Koordinaten ist:

$$
\hat{\mu} = q \hat{x}
$$

Wobei $q$ eine Art Übergängsladung darstellt, aber für die Betrachtung hier entfallen kann.

- Zeigen Sie, dass Übergänge zwischen zwei Zuständen nicht erlaubt sind, wenn $\Delta n$ = 2.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.special import hermite, factorial
from ipywidgets import interact
plt.xkcd()
plt.style.use("seaborn-ticks")
plt.rcParams["legend.handlelength"] = 0.8
plt.rcParams["legend.framealpha"] = 0.8
plt.rcParams.update({"font.size": 12})

a0 = 1
hbar = 1
m_e = 1
k = 2
w = (hbar**2 / (m_e * k))
x = np.linspace(-5,5,10000) * a0

def wavefunction(x, n, k):
    alpha = (hbar ** 2 / (m_e * k))**(1/4)
    x = x / alpha
    C = np.sqrt(1 / (2**n * factorial(n) * np.sqrt(np.pi) * alpha))
    H = hermite(n, monic=True)
    H = H(x)
    gauss = np.exp(-(x**2) / 2)
    return C * H * gauss

from scipy import integrate
def int_psi_mu_psi(x, n1, n2):
    psi_mu_psi = wavefunction(x, n1, k) * x / (hbar ** 2 / (m_e * k))**(1/4) * wavefunction(x, n2, k)
    integral = integrate.trapz(psi_mu_psi, x)
    return integral

def show_plot(n1=0, n2=1):
    if n1 == n2:
        print("$\Delta n$ must be bigger than 0.")
        return None
    fig, ax = plt.subplots(1,3 ,figsize=(15,5), sharey=True, sharex=True)
    ax[0].plot(x, wavefunction(x, n1, k), color="crimson", label="$\psi_{{{}}}$".format(n1))
    ax[0].plot(x, wavefunction(x, n2, k), color="orange", label="$\psi_{{{}}}$".format(n2))
    ax[0].set_title("$\psi_{n_1}$")
    ax[1].plot(x, wavefunction(x, n2, k) * x * (hbar ** 2 / (m_e * k))**(-1/4), color="royalblue", label="$\hat{{\mu}}\psi_{{{}}}$".format(n2))
    ax[1].set_title("$\hat{\mu} \psi_{n_2}$")
    psi_mu_psi = wavefunction(x, n1, k) * x * (hbar ** 2 / (m_e * k))**(-1/4) * wavefunction(x, n2,k)
    ax[2].plot(x, psi_mu_psi, color="black", label="$\psi_{{{}}}\hat{{\mu}}\psi_{{{}}}$".format(n1, n2))
    ax[2].fill_between(x, (psi_mu_psi > 0)* psi_mu_psi, color="crimson", alpha=0.5)
    ax[2].fill_between(x, (psi_mu_psi < 0)* psi_mu_psi, color="royalblue", alpha=0.5)
    if abs(int_psi_mu_psi(x, n1, n2)) > 1e-15:        
        ax[2].set_title(r"$< \psi_{n_1} | \hat{\mu} | \psi_{n_2} > \neq 0$")
    else:
        ax[2].set_title(r"$< \psi_{n_1} | \hat{\mu} | \psi_{n_2} > = 0$")
    for axk in range(3):
        ax[axk].legend()
        ax[axk].set_xlabel("x")
        ax[axk].set_yticks([])
        ax[axk].set_xticks([])
    plt.show()

interact(show_plot, n1=(0,3,1), n2=(0,3,1))

## Molekulare Schwingungszustände

Das Potential von zwei zueinander schwingenden Teilchen verhält sich nicht grundsätzlich wie ein Harmonischer Oszillator: Auf kurze Distanzen stoßen die Teilchen sich ab, auf mittlere Distanzen ziehen sie sich an, und auf lange Distanzen gibt es keine Wechselwirkung mehr.

Es gibt verschiedene physikalisch motivierte Potentiale, die dieses Verhalten richtig beschreiben:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
plt.xkcd()
plt.style.use("seaborn-ticks")
plt.rcParams["legend.handlelength"] = 0.8
plt.rcParams["legend.framealpha"] = 0.8
plt.rcParams.update({"font.size": 12})

def morse(x, De, k, re):
    return De * (1 - np.exp(-np.sqrt(k/(2*De))*(x-re)))**2

def buckingham(A = 6, B = 0.75, C = 1, r = np.linspace(.8,4,100), e = 2.71828):
    size = len(r) 
    v_Buck = np.ones(size)
    for i in range(len(r)):
        v_one = (A) 
        v_two = ((e)**((-B)*(r[i]))) 
        v_three = ((C)/((r[i])**6)) 
        v_Buck[i] = -((v_one)*(v_two)-(v_three))
    return r, v_Buck

def lj(r, epsilon=1.0, sigma=1.0):
    if r > 0.0:
        return epsilon*(sigma**12/r**12-sigma**6/r**6)
    else:
        return None
    
xm = np.linspace(0, 10, 200)
xlj = np.linspace(0.5, 2.5, 100)
vlj = np.vectorize(lj)
r, v_output=buckingham() 

fig, ax = plt.subplots(1, 3,figsize=(18,5), sharey=False, sharex=False)
ax[0].plot(xm, morse(xm, 1, 1.2, 1), color="blue")
ax[0].set_title("Morse: $De \cdot (1 - e^{-\sqrt{k/2De} \cdot x-r_e})^2$")
ax[1].plot(xlj, vlj(xlj, 1, 1), color="red")
ax[1].set_title("Lennard-Jones (12, 6): $\epsilon \cdot ( (\sigma/r)^{12} - (\sigma/r)^6)$")
ax[1].set_xlim([0.9, 2.0])
ax[1].set_ylim([-0.28, 0.1])
ax[2].plot(r, v_output, color="green")
ax[2].set_title("Buckingham: $ A \cdot e^{-Br} - C/r^6$")
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[2].set_xticks([])
ax[2].set_yticks([])
plt.show()

Diese Potentiale enthalten allerdings Parameter, die nicht immer leicht zugänglich sind. Das Lösen der Schrödinger-Gleichung für diese Potentiale ist schwierig.

Um das Minimum herum aber können wir die Potentiale mit einer Taylor-Expansion darstellen:

$$
V(r) \approx V(r_0) + \left(\frac{\partial V(x)}{\partial r}\right)_{r=r_0} \cdot r ~
+~ \frac{1}{2} \left(\frac{\partial^2 V(x)}{\partial r^2}\right)_{r=r_0} \cdot r^2 + \ldots
$$

Hierbei ist $V(r_0)$ die Potentialtiefe, die willkürlich verschoben werden kann. Die erste Ableitung entfällt, da wir uns am Minimum befinden. Für kleine Auslenkungen von $r$ können alles nach dem zweiten Term vernachlässigen:

$$
V(r) \approx \frac{1}{2} \left(\frac{\partial^2 V(x)}{\partial r^2}\right)_{r=r_0} \cdot r^2 = \frac{1}{2} k r^2
$$

Dabei handelt es sich um dasselbe harmonische Potential wie im harmonischen Oszillator.

Wenn wir nun in einem relativen Koordinatensystem arbeiten, in welchem eines der zwei Teilchen als fixiert betrachtet wird und das zweite mit einer reduzierten Masse $\mu = \frac{m_1 m_2}{m_1 + m_2}$ schwingt, lautet die Schrödinger-Gleichung:

$$
\frac{- \hbar^2}{2 \mu} \frac{\partial^2 \Psi}{\partial r^2} + \frac{1}{2} k r^2 \Psi = E \Psi
$$

Diese ist identisch zum Schrödinger-Problem des Harmonischen Oszillators. Für kleine Auslenkungen können wir ein System zweier schwingender Teilchen also wie ein effektives Teilchen in einem parabolischen Potential lösen. Dies nennt man die **harmonische Näherung**.

## Harmonische Näherung am Morse-Potential

Das Morse-Potential hat folgende analytische Form:

$$
V(r) = D_e \left(1 - e^{-\alpha(r-r_0)}\right)^2
$$

Hierbei beschreibt $D_e$ die Dissoziationsenergie, also das Minimum der Potentialkurve bezüglich des Dissoziationslimits, $r_0$ ist der Gleichgewichtsabstand und $\alpha$ ist ein Parameter, der die Krümmung der Kurve bestimmt.

Der Einfluss der Parameter ist hier gezeigt:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import interact

plt.xkcd()
plt.style.use("seaborn-ticks")
plt.rcParams["legend.handlelength"] = 0.8
plt.rcParams["legend.framealpha"] = 0.8
plt.rcParams.update({"font.size": 12})

def morse(x, De, alpha, re):
    return De * (1 - np.exp(-alpha*(x-re)))**2

x = np.linspace(0, 4, 200)

De = -(460.6593 - 0.4449 - 460.513)*27.2114
alpha = 1.8
re = 1.275

def show_plot(D_e=0.8, Alpha=0.6, R_e=0.7):
    DE = D_e * De
    ALPHA = Alpha * alpha
    RE = R_e * re
    fig, ax = plt.subplots(1, 1, figsize=(5,5), sharey=True, sharex=True)
    ax.set_ylim([-0.5, 12])
    ax.axvline(RE, 0, 12, color="gray", linestyle="--", alpha=0.5)
    ax.axhline(DE, np.min(x), np.max(x), color="crimson", linestyle="--", alpha=0.5)
    ax.plot(x, morse(x, DE, ALPHA, RE), alpha=0.6, color="blue")
    ax.set_title("Morse potential")
    ax.set_xlabel("r")
    ax.set_ylabel("$E - D_e$")
    
interact(show_plot, D_e=(0.8, 1.2, 0.1), Alpha=(0.6, 1.2, 0.1), R_e=(0.7, 1.5, 0.1))

Wir führen jetzt die harmonische Näherung am Morse-Potential ein:

$$
V(r) \approx \frac{1}{2} \left(\frac{\partial^2}{\partial r^2} D_e \left(1 - e^{-\alpha(r-r_0)}\right)^2 \right)_{r=r_0} \cdot r^2 
$$

Schauen wir uns zuerst die erste Ableitung an:

$$
\begin{aligned}
V'(r) &= D_e \frac{\partial}{\partial r} \left(1 - e^{-\alpha(r-r_0)}\right)^2 \\
&= D_e \cdot \frac{\partial}{\partial r} 
\left( 1 - 2 e^{-\alpha(r-r_0)} + e^{- 2 \alpha(r-r_0)} \right) \\
&= 2 D_e \alpha \left(e^{-\alpha(r-r_0)} - e^{2 \cdot\alpha(r-r_0)} \right)
\end{aligned}
$$

Dann die zweite Ableitung:

$$
\begin{aligned}
V''(r) &= 2 D_e \alpha \frac{\partial}{\partial r}
\left(e^{-\alpha(r-r_0)} - e^{2 \cdot\alpha(r-r_0)} \right) \\
&= -2 D_e \alpha^2 \left( e^{-\alpha(r-r_0)} - 2 e^{2 \cdot\alpha(r-r_0)}\right)
\end{aligned}
$$

Dann setzten wir die Stelle $r=r_0$ ein und erhalten:

$$
\begin{aligned}
V''(r_0) &= -2 D_e \alpha^2 \left(e^{0} - 2 e^{0}\right) \\
&= 2 D_e \alpha^2
\end{aligned}
$$

Dieser Ausdruck entspricht der Kraftkonstante $k$ aus dem harmonischen Potential.

Daraus können wir die Schwingungsfrequenz $\nu$ des HCl-Moleküls bestimmen:

$$
\begin{aligned}
k &= 2 D_e \alpha^2 = 2* 1.8^2 \mathring A^{-2} \cdot \left(-0.45 -460.05 +460.66  \right) E_H \\
&= 1.057 \frac{E_H}{\mathring A^{2}} = 460.6 \frac{N}{m} \\ 
~~\\
\mu &= \frac{m_H m_{Cl}}{m_H + m_{Cl}} = \frac{1}{m_H} + \frac{1}{m_{Cl}} = \frac{35}{36} g/mol \\
&= \frac{35}{36} \cdot 10^{-3} kg / N_A = 1.615 \cdot 10^{-27} kg\\
~~ \\
\omega &= \sqrt{\frac{k}{\mu}} = \sqrt{\frac{460~N/m}{1.615 \cdot 10^{-27} kg}} = 5.338 \cdot 10^{14} s^{-1}\\
~~\\
\tilde{\nu} &= \frac{\omega}{2 \pi c} = \frac{5.338 \cdot 10^{14} s^{-1}}{2 \pi \cdot 299792458 \cdot 10^2 cm/s}
= \mathbf{2834~cm^{-1}}
\end{aligned}
$$

# Rovibronische Spektren

Molekulare Schwingungen finden typischerweise auf einer Zeitskala von wenigen zehn bis hundert Femtosekunden statt. Auf derselben Zeitskala passieren molekulare Rotationen, sodass diese miteinander koppeln.

Die Lösungen des starren Rotators sind Kugelflächenfunktionen. Die Energieniveaus sind gequantelt nach:

$$
E_J = h c BJ(J+1)
$$

Hierbei wird die Rotationsquantenzahl $J = 0, 1, 2...$ verwendet, um diese von der Bahndrehimpulsquantenzahl $l$ der Lösungen des Wasserstoffatoms zu unterscheiden, obwohl diese genau die gleiche Bedeutung hat. In der Rotationskonstante $B$ sind mehrere Konstanten vereinigt:

$$
B = \frac{h}{8\pi^2 c I}
$$

Wobei $I = \mu r^2$ das Trägheitsmoment mit der reduzierten Masse ist.

Für den sphärischen und linearen Rotor sind nur Übergänge zu sehen, die folgende Bedingung erfüllen:

$$
\tilde{\nu} = 2 B(J+1)
$$

Die Änderung eines Schwingungszustands geht typischerweise mit der Änderung des Rotationszustands einher. Im sogenannten P-zweig beobachtet man Übergänge von $J \rightarrow J -1$, im R-Zweig Übergänge von $J \rightarrow J +1$. Nur für Systeme geringerer Symmetrie (z.B. paramagnetische Moleküle) beobachtet man auch Übergänge von $ J \rightarrow J$, also wo $\Delta J = 0$, den sogenannten Q-zweig.

Sowohl für den P- und R-Zweig gilt:

$$
\Delta\tilde{\nu} = 2 B(J+1 \pm 1) - 2B(J+1) = \pm2B
$$

## Rotationsspektrum von HCl

<img src="../Pictures/hcl_infrared.PNG" alt="Drawing" style="width: 1000px;"/>

Im HCl-Molekül wird ein Rotationsübergang von $J_0=0$ zu $J_1=1$ bei 20.793 $cm^{-1}$ beobachtet. Gesucht ist der Gleichgewichtsabstand:

$$
\begin{aligned}
I &= \mu r^2 = \frac{m_{Cl}\cdot m_H}{m_{Cl} + m_H} r^2 \\
~~\\
\tilde{\nu} &= 2 B \\
~~\\
B &= \frac{\tilde{\nu}}{2}\\
~~\\
B &= \frac{h}{8\pi^2 c I} = \frac{h}{8\pi^2 c \mu r^2} \\
~~\\
r &= \sqrt{\frac{h}{8 \pi^2 c \mu B}} = \sqrt{\frac{h \cdot 2}{8 \pi^2 c \mu \tilde{\nu}}}
\end{aligned}
$$

Wenn wir annehmen, dass die HCl$^{37}$-Isotopvariante denselben Gleichgewichtsabstand hat, können wir dadurch dessen Wellenzahl berechnen:

$$
\begin{aligned}
\tilde{\nu} &= 2 B = \frac{2 \cdot h}{8\pi^2 c \mu \cdot r^2}
\end{aligned}
$$

Wenn wir also den Rotationsübergang zwischen HCl$^{35}$ und HCl$^{37}$ vergleichen, liegt der Unterschied in der Wellenzahl gerade mal bei 0.032 $cm^{-1}$. Für den Schwingungsübergang bei ca. 2900 $cm^{-1}$ aber gilt:

$$
\begin{aligned}
\tilde{\nu} &= \sqrt{\frac{k}{\mu}} \cdot \frac{1}{2\pi c} \\
\end{aligned}
$$

Somit ist der Verhältnis der verschiedenen Schwingungsfrequenzen proportional zu $\sqrt{\frac{\mu_{HCl^{37}}}{\mu_{HCl^{35}}}}$. Dieses ist etwa 99.925 \%, sodass der Schwingungsübergang sich zu ca. 2.898 $cm^{-1}$ verschiebt, was man auch so beobachtet.